In [1]:
import pandas as pd
import numpy as np
import warnings
import glob
import re
import datetime
import calendar

#### Inputs para Ejecución del Proceso

In [2]:
# Solicitar al usuario el mes y año de cierre
mes_cierre = int(input("Indique el mes de cierre (MM): "))
año_cierre = int(input("Indique el año de cierre (AAAA): "))

# Funcion para generar la fecha con el ultimo dia del mes de cierre
def ultimo_dia_del_mes(mes_cierre, año_cierre):
    ultimo_dia = calendar.monthrange(año_cierre, mes_cierre)[1]
    fecha = datetime.date(año_cierre, mes_cierre, ultimo_dia)
    return fecha.strftime("%d%m%Y")

ultimo_dia_del_mes = ultimo_dia_del_mes(mes_cierre, año_cierre)
# Ejemplo de uso
print(ultimo_dia_del_mes)

30062024


#### Definición de Párametros Iniciales

In [3]:
# Rango de tolerancia para eliminación de registros con valor muy pequeño
tolerancia_eliminacon_cero = 0.5
desde_eliminacon_cero = -tolerancia_eliminacon_cero
hasta_eliminacon_cero = tolerancia_eliminacon_cero

# Rango de tolerancia para eliminación de pólizas cerradas
tolerancia_filtros_iniciales = 10
desde = -tolerancia_filtros_iniciales
hasta = tolerancia_filtros_iniciales

# Rango de tolerancia para ajustar pólizas en plantilla ajustes financieros
tolerancia_ajustes_financieros_desde = 10
tolerancia_ajustes_financieros_hasta = 1000
desde_positivo = tolerancia_ajustes_financieros_desde
desde_negativo = -tolerancia_ajustes_financieros_desde
hasta_positivo = tolerancia_ajustes_financieros_hasta
hasta_negativo = -tolerancia_ajustes_financieros_hasta

valor_no_ajuste = 6000
desde_no_ajuste = -valor_no_ajuste
hasta_no_ajuste = valor_no_ajuste

# Rango de tolerancia para ajustar pólizas en dólares
tolerancia_porc_pendientes = 0.1
desde_porc_pendientes = -tolerancia_porc_pendientes
hasta_porc_pendientes = tolerancia_porc_pendientes

tipos_fuente = pd.read_excel('01. Input/Tipos Texto.xlsx')

# Lista de frases para identificar las provisiones
provision = tipos_fuente.loc[tipos_fuente['TIPO'] == "Provision"]['TEXTO'].tolist()

# Lista de frases para identificar los pagos
pago = tipos_fuente.loc[tipos_fuente['TIPO'] == "Pago"]['TEXTO'].tolist()

# Lista de frases para identificar los ajustes de provision
ajuste_provision = tipos_fuente.loc[tipos_fuente['TIPO'] == "Ajuste_Provision"]['TEXTO'].tolist()

# Lista de frases para identificar los ajustes de pago
ajuste_pago = tipos_fuente.loc[tipos_fuente['TIPO'] == "Ajuste_Pago"]['TEXTO'].tolist()

# Lista CDTIPO_FUENTE completa
cdtipo_fuente = tipos_fuente['TEXTO'].tolist()

#### Importación de Archivos

In [4]:
# Archivos CSV a concatenar
all_files_vida = glob.glob('01. Input/02. Vida/*.txt')

# Esquema de datos
cabecera ={"CDSOCIEDAD": "object",
            "CDRAMO": "object",
            "NMPOLIZA": "object",
            "NMRECIBO": "object",
            "CDCONCEPTO": "object",
            "CDCOASEGURO": "object",
            "CDAGENTE": "object",
            "CDDELEGACION": "object",
            "PTPRIMA": "float",
            "PTCOMISION": "float",
            "POCOMISION": "float",
            "CDTIPO_FUENTE": "object",
            "FEDOCUMENTO": "object",
            "FEREGISTRO": "object",
            "FEINI_VIGENCIA": "object",
            "FEFIN_FIGENCIA": "object",
            "CDLIBRO":"object",
            "VACIO":"object"}

# Lista para almacenar los DataFrames de los archivos CSV
dataframes_vida = []

# Cargar cada archivo CSV en un DataFrame y agregarlo a la lista
for file in all_files_vida:
    warnings.filterwarnings('ignore')
    df = pd.read_table(file, sep=';', header=None, names=list(cabecera.keys()), dtype=cabecera)
    dataframes_vida.append(df)
    print(f'Archivo {file} cargado. Tiene {df.shape[0]} filas')

# Archivo principal Vida
vida = pd.concat(dataframes_vida, ignore_index=True)
vida = vida.drop('VACIO', axis=1)
vida = vida.loc[vida['PTCOMISION'].notnull()]
vida = vida.loc[~(vida['PTCOMISION'].between(desde_eliminacon_cero, hasta_eliminacon_cero, inclusive='both'))]
vida = vida.fillna("")

Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_6106111.txt cargado. Tiene 799999 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061110.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061111.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061112.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061113.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061114.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061115.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061116.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61061117.txt cargado. Tiene 800000 filas
Archivo 01. Input/02. Vida\libro_auxiliar_prov_comisiones_vida_61

In [5]:
vida['PTCOMISION'].sum()

np.float64(78296578105.33018)

In [6]:
## Insumos varios
dolares = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='Dolares', dtype=str)
retirados = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='Retirados', dtype={'CDAGENTE':'str'})
codigos_directos = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='Codigos directos', dtype=str)
asesores_formacion = pd.read_excel('01. Input/Insumos Comisiones.xlsx', sheet_name='FyD', dtype={'CDAGENTE':'str'})

# Transformación Insumos
dolares['DOLARES'] = "X"
retirados['RETIRADOS'] = "X"
codigos_directos['CDDIR'] = "X"
asesores_formacion['FYD'] = "X"

# Ajustar NMPOLIZA en el archivo dolares para que tenga como mínimo 12 dígitos
dolares['NMPOLIZA'] = np.select(
    [dolares['NMPOLIZA'].str.len() < 12],
    [dolares['NMPOLIZA'].str.rjust(12, "0")],
    default=dolares['NMPOLIZA']
)

# Seleccionar columnas necesarias en el archivo retirados
retirados = retirados[['CDAGENTE', 'RETIRADOS', 'FEBAJA']]

#### Validación CDTIPO_FUENTE

In [7]:
cdtipo_fuente_validar = vida['CDTIPO_FUENTE'].unique()

# Crear una lista de elementos que están en cdtipo_fuente_validar pero no en cdtipo_fuente
elementos_no_encontrados = [elemento for elemento in cdtipo_fuente_validar if elemento not in cdtipo_fuente]

# Verificar si hay elementos no encontrados
if elementos_no_encontrados:
    print("Los siguientes elementos no se encuentran en cdtipo_fuente:")
    for elemento in elementos_no_encontrados:
        print(elemento)
else:
    print("Todos los elementos de cdtipo_fuente_validar se encuentran en cdtipo_fuente.")


Todos los elementos de cdtipo_fuente_validar se encuentran en cdtipo_fuente.


#### Transformaciones Iniciales

In [8]:
# Ajustar NMPOLIZA para que tenga como mínimo 12 dígitos
vida['NMPOLIZA'] = np.select(
    [vida['NMPOLIZA'].str.len() < 12],
    [vida['NMPOLIZA'].str.rjust(12, "0")],
    default=vida['NMPOLIZA']
)

# Ajustar CDRAMO para que tenga como mínimo 3 dígitos
vida['CDRAMO'] = np.select(
    [vida['CDRAMO'].str.len() < 3],
    [vida['CDRAMO'].str.rjust(3, "0")],
    default=vida['CDRAMO']
)

# Quitar letras y caracteres especiales de NMRECIBO y convertir a número
vida['NMRECIBO'] = vida['NMRECIBO'].apply(lambda x: re.sub(r'[^0-9]', '', x))
vida['NMRECIBO'] = pd.to_numeric(vida['NMRECIBO'], errors='coerce')

# Agregar tipo M a CDCOASEGURO
vida['CDCOASEGURO'] = np.select(
    [vida['CDTIPO_FUENTE'].isin(ajuste_pago),
     (vida['CDCOASEGURO'] == "") & (vida['CDTIPO_FUENTE'].isin(pago))], 
    ["M",
     "M"],
    default=vida['CDCOASEGURO']
)

#### Depuración de Tablas

In [9]:
## Eliminación de Pólizas con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza para determinar su valor y marcar las que tienen valor 0
vida_depurada1 = vida.groupby('NMPOLIZA')['PTCOMISION'].sum().reset_index()
vida_depurada1['ELIMINAR?'] = np.select(
    [vida_depurada1['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada1 = vida_depurada1.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas con valor 0
vida = vida.merge(vida_depurada1, on='NMPOLIZA', how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

In [10]:
## Eliminación de Pólizas/Recibo con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza/recibo para determinar su valor y marcar las que tienen valor 0
vida_depurada2 = vida[(vida['NMRECIBO'] != "99999999") | (vida['NMRECIBO'] != "9999999") | (vida['NMRECIBO'] != "999999") | (vida['CDCOASEGURO']) != "M"]
vida_depurada2 = vida_depurada2.groupby(['NMPOLIZA','NMRECIBO'])['PTCOMISION'].sum().reset_index()
vida_depurada2['ELIMINAR?'] = np.select(
    [vida_depurada2['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada2 = vida_depurada2.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/recibo con valor 0
vida = vida.merge(vida_depurada2, on=['NMPOLIZA','NMRECIBO'], how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

In [11]:
## Eliminación de Pólizas/Agente con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza/agente para determinar su valor y marcar las que tienen valor 0
vida_depurada3 = vida[(vida['NMRECIBO'] != "99999999") | (vida['NMRECIBO'] != "9999999") | (vida['NMRECIBO'] != "999999") | (vida['CDCOASEGURO']) != "M"]
vida_depurada3 = vida_depurada3.groupby(['NMPOLIZA','CDAGENTE'])['PTCOMISION'].sum().reset_index()
vida_depurada3['ELIMINAR?'] = np.select(
    [vida_depurada3['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada3 = vida_depurada3.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/agente con valor 0
vida = vida.merge(vida_depurada3, on=['NMPOLIZA','CDAGENTE'], how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

In [12]:
## Identificación de registros "Provisión" para agruparlos con sus respectivos "Pagos" y eliminar si su valor es 0 (Tolerancia definida en parametros iniciales)
                                                 
# Marcar los registros que estén dentro de la lista de opciones de provisión
vida['TIPO'] = np.select(
    [vida['CDTIPO_FUENTE'].isin(provision)],
    ["Provision"],
    default="Pago"
)

vida['OBSERVACION'] = ""

# Se ordena el dataset por póliza y recibo 
vida.sort_values(by=['NMPOLIZA', 'NMRECIBO', 'TIPO'], ascending=[True, True, False], inplace=True)

# Asignación de un número único de identificación a cada provisión
vida['PROVISION_FLAG'] = np.where(vida['TIPO'] == "Provision", 1, 0)
vida['CDPROVISION'] = np.where(vida['PROVISION_FLAG'] == 1, vida['PROVISION_FLAG'].cumsum(), "")
vida.drop(['PROVISION_FLAG'], axis=1, inplace=True)

#####################################################################################################################
Nmpoliza = ""
Tipo = ""
cdprovision = "" # Va a guardar el número de provisión anterior
Aux_cdprovision = "" # Va a guardar el contenido de la línea anterior en el campo 

for index, row in vida.iterrows():
    # Escenario primera línea
    if Nmpoliza == "" and Tipo == "":
        Nmpoliza = row['NMPOLIZA']
        Tipo = row['TIPO']
        if Tipo == "Provision":
            #print("Se ejecuto primera linea")
            cdprovision = row['CDPROVISION']
            vida.at[index, "OBSERVACION"] = ""
        else:
            vida.at[index, "OBSERVACION"] = "Pago sin Provision"
    # Escenarios para misma Nmpoliza
    elif row['NMPOLIZA'] == Nmpoliza:
        #print(f"Registro con la poliza igual al anterior;Actual {row['NMPOLIZA']}, anterior: {Nmpoliza}")
        # Más de una provisión consecutiva dentro de una misma Nmpoliza
        if row['CDPROVISION'] != "" and Tipo == "Provision":
            vida.at[index, "CDPROVISION"] = Aux_cdprovision
            vida.at[index, "OBSERVACION"] = ""

        # Cuando encuentra por primera vez una provisión despues de pagos
        # No debería pasar si se organiza cdprovision descendente
        elif row['CDPROVISION'] != "" and cdprovision == "":
            cdprovision = row['CDPROVISION']
            vida.at[index, "OBSERVACION"] = ""

        # Cuando son pagos en una misma Nmpoliza y no tienen una provisión previa
        elif cdprovision == "" and Tipo == "Pago":
             vida.at[index, "OBSERVACION"] = "Pago sin Provision"
        
        # Cuando es un pago despues de una provión
        elif row['CDPROVISION'] == "" and row['TIPO'] == "Pago":
            vida.at[index, "CDPROVISION"] = cdprovision
            vida.at[index, "OBSERVACION"] = ""
        
        # Encontro otra provisión pero el anterior registro es un pago
        elif row['TIPO'] != Tipo and row['TIPO'] == "Provision":
            cdprovision = row['CDPROVISION']
            vida.at[index, "OBSERVACION"] = ""

        # Acá solo entran los registros que no tienen codigo de provisión y son Provisiones
        else:
            #print(f"Valor tabla para CDPROVISION: {row['CDPROVISION']}, Valor tabla NMPOLIZA: {row['TIPO']}")
            vida.at[index, "OBSERVACION"] = "Revisar"
            
    elif row['NMPOLIZA'] != Nmpoliza:
        #print(f"Registro con la poliza DIFERENTE al anterior;Actual {row['NMPOLIZA']}, anterior: {Nmpoliza}")
        Nmpoliza = row['NMPOLIZA']
        cdprovision = ""
        if row['TIPO'] == "Pago":
            vida.at[index, "OBSERVACION"] = "Pago sin Provision"
        elif row['TIPO'] == "Provision":
            cdprovision = row['CDPROVISION']
    
    # Variables del registro anterior
    Tipo = row['TIPO']
    Aux_cdprovision = vida.at[index, "CDPROVISION"]
#####################################################################################################################
    
# Quitar código de provisión a las provisiones con recibo 99999999
vida['CDPROVISION'] = np.where((vida['NMRECIBO'].fillna("") == "99999999") | (vida['NMRECIBO'].fillna("") == "999999"),"",vida['CDPROVISION'])

# Edición columna Observación para marcar polizas 99999999 y 999999 como revisión
#vida['OBSERVACION'] = np.where((vida['NMRECIBO'].fillna("") == "99999999") | (vida['NMRECIBO'].fillna("") == "999999"),"Revisar",vida['OBSERVACION'])

# Agrupar pólizas por código de provisión para determinar su valor y marcar las que tienen valor 0 
vida_depurada4 = vida.groupby('CDPROVISION').agg({'PTCOMISION': 'sum'}).reset_index()

vida_depurada4['ELIMINAR?'] = np.select(
    [vida_depurada4['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada4 = vida_depurada4.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas por código de provisión con valor 0
vida = vida.merge(vida_depurada4, on=['CDPROVISION'], how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

In [13]:
## Eliminación de Pólizas por código coaseguro con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por código coaseguro para determinar su valor y marcar las que tienen valor 0
vida_depurada5 = vida.loc[vida['CDCOASEGURO'] == "M"]
vida_depurada5 = vida_depurada5.groupby(['NMPOLIZA','CDCOASEGURO','CDAGENTE','FEREGISTRO'])['PTCOMISION'].sum().reset_index()
vida_depurada5['ELIMINAR?'] = np.select(
    [vida_depurada5['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada5 = vida_depurada5.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/agente con valor 0
vida = vida.merge(vida_depurada5, on=['NMPOLIZA','CDCOASEGURO','CDAGENTE','FEREGISTRO'], how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

In [14]:
## Eliminación de Pólizas/Recibo con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza/recibo para determinar su valor y marcar las que tienen valor 0
vida_depurada6 = vida[(vida['NMRECIBO'] != "99999999") | (vida['NMRECIBO'] != "9999999") | (vida['NMRECIBO'] != "999999") | (vida['CDCOASEGURO']) != "M"]
vida_depurada6 = vida_depurada6.groupby(['NMPOLIZA','NMRECIBO'])['PTCOMISION'].sum().reset_index()
vida_depurada6['ELIMINAR?'] = np.select(
    [vida_depurada6['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada6 = vida_depurada6.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas/recibo con valor 0
vida = vida.merge(vida_depurada6, on=['NMPOLIZA','NMRECIBO'], how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

In [15]:
## Eliminación de Pólizas con valor 0 (Tolerancia definida en parametros iniciales)

# Agrupar por póliza para determinar su valor y marcar las que tienen valor 0
vida_depurada7 = vida.groupby('NMPOLIZA')['PTCOMISION'].sum().reset_index()
vida_depurada7['ELIMINAR?'] = np.select(
    [vida_depurada7['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["X"],
    default=""
)
vida_depurada7 = vida_depurada7.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para eliminar los registros de las pólizas con valor 0
vida = vida.merge(vida_depurada7, on='NMPOLIZA', how='left')
vida = vida.loc[vida['ELIMINAR?'] != "X"]
vida = vida.drop('ELIMINAR?', axis=1)

#### Identificación para Ajustes Financieros

In [16]:
## Identificación de registros para ajustes financieros por póliza

# Agrupar por póliza para determinar su valor y marcar las que tienen valor apto para ajuste
vida_ajustes1_temp = vida.groupby(['NMPOLIZA'])['PTCOMISION'].sum().reset_index()
vida_ajustes1_temp['AJUSTAR?'] = np.select(
    [((vida_ajustes1_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (vida_ajustes1_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado Póliza"],
    default=""
)
vida_ajustes1_temp = vida_ajustes1_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
vida_ajustes1 = vida.merge(vida_ajustes1_temp, on=['NMPOLIZA'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
vida_ajustes1['REV_AJUSTE'] = np.select(
    [(vida_ajustes1['AJUSTAR?'] == "Ajustado Póliza") & (vida_ajustes1['CDTIPO_FUENTE'].isin(provision)) & (vida_ajustes1['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
vida_rev_ajustes1 = vida_ajustes1.loc[vida_ajustes1['REV_AJUSTE'] == "NO"]
vida_rev_ajustes1 = vida_rev_ajustes1.groupby(['NMPOLIZA', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
vida_rev_ajustes1 = vida_rev_ajustes1.drop('CUENTA_REV_AJUSTE', axis=1)
vida_ajustes1 = vida_ajustes1.drop('REV_AJUSTE', axis=1)
vida_ajustes1 = vida_ajustes1.merge(vida_rev_ajustes1, on=['NMPOLIZA'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
vida_ajustes1['AJUSTAR?'] = np.select(
    [(vida_ajustes1['AJUSTAR?'] == "Ajustado Póliza") & (vida_ajustes1['REV_AJUSTE'] == "NO")],
    [""],
    default=vida_ajustes1['AJUSTAR?']
)
vida_ajustes1 = vida_ajustes1.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
vida_ajustados1 = vida_ajustes1.loc[vida_ajustes1['AJUSTAR?'] == "Ajustado Póliza"]
vida_revision1 = vida_ajustes1.loc[vida_ajustes1['AJUSTAR?'] != "Ajustado Póliza"]

In [17]:
## Identificación de registros para ajustes financieros por póliza/recibo

vida_revision1 = vida_revision1.drop('AJUSTAR?', axis=1)

# Agrupar por póliza/recibo para determinar su valor y marcar las que tienen valor apto para ajuste
vida_ajustes2_temp = vida_revision1.groupby(['NMPOLIZA', 'NMRECIBO'])['PTCOMISION'].sum().reset_index()
vida_ajustes2_temp['AJUSTAR?'] = np.select(
    [((vida_ajustes2_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (vida_ajustes2_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado Póliza/Recibo"],
    default=""
)
vida_ajustes2_temp = vida_ajustes2_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
vida_ajustes2 = vida_revision1.merge(vida_ajustes2_temp, on=['NMPOLIZA', 'NMRECIBO'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
vida_ajustes2['REV_AJUSTE'] = np.select(
    [(vida_ajustes2['AJUSTAR?'] == "Ajustado Póliza/Recibo") & (vida_ajustes2['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
vida_rev_ajustes2 = vida_ajustes2.loc[vida_ajustes2['REV_AJUSTE'] == "NO"]
vida_rev_ajustes2 = vida_rev_ajustes2.groupby(['NMPOLIZA', 'NMRECIBO', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
vida_rev_ajustes2 = vida_rev_ajustes2.drop('CUENTA_REV_AJUSTE', axis=1)
vida_ajustes2 = vida_ajustes2.drop('REV_AJUSTE', axis=1)
vida_ajustes2 = vida_ajustes2.merge(vida_rev_ajustes2, on=['NMPOLIZA', 'NMRECIBO'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
vida_ajustes2['AJUSTAR?'] = np.select(
    [(vida_ajustes2['AJUSTAR?'] == "Ajustado Póliza/Recibo") & (vida_ajustes2['REV_AJUSTE'] == "NO")],
    [""],
    default=vida_ajustes2['AJUSTAR?']
)
vida_ajustes2 = vida_ajustes2.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
vida_ajustados2 = vida_ajustes2.loc[vida_ajustes2['AJUSTAR?'] == "Ajustado Póliza/Recibo"]
vida_revision2 = vida_ajustes2.loc[vida_ajustes2['AJUSTAR?'] != "Ajustado Póliza/Recibo"]

In [18]:
## Identificación de registros para ajustes financieros por póliza/agente

vida_revision2 = vida_revision2.drop('AJUSTAR?', axis=1)

# Agrupar por póliza/agente para determinar su valor y marcar las que tienen valor apto para ajuste
vida_ajustes3_temp = vida_revision2.groupby(['NMPOLIZA', 'CDAGENTE'])['PTCOMISION'].sum().reset_index()
vida_ajustes3_temp['AJUSTAR?'] = np.select(
    [((vida_ajustes3_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (vida_ajustes3_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado Póliza/Agente"],
    default=""
)
vida_ajustes3_temp = vida_ajustes3_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
vida_ajustes3 = vida_revision2.merge(vida_ajustes3_temp, on=['NMPOLIZA', 'CDAGENTE'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
vida_ajustes3['REV_AJUSTE'] = np.select(
    [(vida_ajustes3['AJUSTAR?'] == "Ajustado Póliza/Agente") & (vida_ajustes3['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
vida_rev_ajustes3 = vida_ajustes3.loc[vida_ajustes3['REV_AJUSTE'] == "NO"]
vida_rev_ajustes3 = vida_rev_ajustes3.groupby(['NMPOLIZA', 'CDAGENTE', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
vida_rev_ajustes3 = vida_rev_ajustes3.drop('CUENTA_REV_AJUSTE', axis=1)
vida_ajustes3 = vida_ajustes3.drop('REV_AJUSTE', axis=1)
vida_ajustes3 = vida_ajustes3.merge(vida_rev_ajustes3, on=['NMPOLIZA', 'CDAGENTE'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
vida_ajustes3['AJUSTAR?'] = np.select(
    [(vida_ajustes3['AJUSTAR?'] == "Ajustado Póliza/Agente") & (vida_ajustes3['REV_AJUSTE'] == "NO")],
    [""],
    default=vida_ajustes3['AJUSTAR?']
)
vida_ajustes3 = vida_ajustes3.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
vida_ajustados3 = vida_ajustes3.loc[vida_ajustes3['AJUSTAR?'] == "Ajustado Póliza/Agente"]
vida_revision3 = vida_ajustes3.loc[vida_ajustes3['AJUSTAR?'] != "Ajustado Póliza/Agente"]

In [19]:
## Identificación de registros para ajustes financieros por código de agrupación

vida_revision3 = vida_revision3.drop('AJUSTAR?', axis=1)

# Agrupar por código de agrupación para determinar su valor y marcar las que tienen valor apto para ajuste
vida_ajustes4_temp = vida_revision3.groupby(['NMPOLIZA', 'CDPROVISION'])['PTCOMISION'].sum().reset_index()
vida_ajustes4_temp['AJUSTAR?'] = np.select(
    [((vida_ajustes4_temp['PTCOMISION'].between(hasta_negativo, desde_negativo, inclusive='both')) | 
     (vida_ajustes4_temp['PTCOMISION'].between(desde_positivo, hasta_positivo, inclusive='both')))],
    ["Ajustado código agrupación"],
    default=""
)
vida_ajustes4_temp = vida_ajustes4_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
vida_ajustes4 = vida_revision3.merge(vida_ajustes4_temp, on=['NMPOLIZA', 'CDPROVISION'], how='left')

# Identificar cuales registros que se marcaron como Ajuste no aplican realmente
vida_ajustes4['REV_AJUSTE'] = np.select(
    [(vida_ajustes4['AJUSTAR?'] == "Ajustado código agrupación") & (vida_ajustes4['PTCOMISION'].between(desde_no_ajuste, hasta_no_ajuste, inclusive='both'))], 
    ["NO"],
    default="SI"
)
vida_rev_ajustes4 = vida_ajustes4.loc[vida_ajustes4['REV_AJUSTE'] == "NO"]
vida_rev_ajustes4 = vida_rev_ajustes4.groupby(['NMPOLIZA', 'CDPROVISION', 'REV_AJUSTE'])['REV_AJUSTE'].count().reset_index(name='CUENTA_REV_AJUSTE')
vida_rev_ajustes4 = vida_rev_ajustes4.drop('CUENTA_REV_AJUSTE', axis=1)
vida_ajustes4 = vida_ajustes4.drop('REV_AJUSTE', axis=1)
vida_ajustes4 = vida_ajustes4.merge(vida_rev_ajustes4, on=['NMPOLIZA', 'CDPROVISION'], how='left')

# Quitar marca de ajuste a las pólizas que tienen registros con valores "pequeños"
vida_ajustes4['AJUSTAR?'] = np.select(
    [(vida_ajustes4['AJUSTAR?'] == "Ajustado código agrupación") & (vida_ajustes4['REV_AJUSTE'] == "NO")],
    [""],
    default=vida_ajustes4['AJUSTAR?']
)
vida_ajustes4 = vida_ajustes4.drop('REV_AJUSTE', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
vida_ajustados4 = vida_ajustes4.loc[vida_ajustes4['AJUSTAR?'] == "Ajustado código agrupación"]
vida_revision4 = vida_ajustes4.loc[vida_ajustes4['AJUSTAR?'] != "Ajustado código agrupación"]

In [20]:
## Identificación de registros para ajustes por recibo duplicado

# Identificar los recibos que se encuentran en pólizas distintas
recibos_duplicados = vida_revision4.groupby(['NMPOLIZA', 'NMRECIBO'])['PTCOMISION'].sum().reset_index()
recibos_duplicados['VALIDACION_DUPLICADO'] = recibos_duplicados.duplicated(subset=['NMRECIBO'], keep=False)
recibos_duplicados = recibos_duplicados.sort_values(by=['NMRECIBO'], ascending=[True])
recibos_duplicados['NMRECIBO_DUPLICADO'] = np.select(
    [recibos_duplicados['VALIDACION_DUPLICADO'] == True],
    ["Recibo Duplicado"],
    default=""
)
recibos_duplicados = recibos_duplicados.loc[recibos_duplicados['NMRECIBO_DUPLICADO'] == "Recibo Duplicado"]

# Agrupar por recibos duplicados para determinar su valor y marcar las que tienen valor apto para ajuste (Tolerancia definida en parametros iniciales)
vida_revision4 = vida_revision4.drop('AJUSTAR?', axis=1)

vida_ajustes5_temp = recibos_duplicados.groupby(['NMRECIBO', 'NMRECIBO_DUPLICADO'])['PTCOMISION'].sum().reset_index()
vida_ajustes5_temp['AJUSTAR?'] = np.select(
    [vida_ajustes5_temp['PTCOMISION'].between(desde, hasta, inclusive='both')],
    ["Ajustado Recibo Duplicado"],
    default=""
)
vida_ajustes5_temp = vida_ajustes5_temp.drop('PTCOMISION', axis=1)

# Cruce de tabla agrupada con la completa para identificar registros que se deben ajustar
vida_ajustes5 = vida_revision4.merge(vida_ajustes5_temp, on=['NMRECIBO'], how='left')
vida_ajustes5['OBSERVACION'] = np.select(
    [vida_ajustes5['NMRECIBO_DUPLICADO'] == "Recibo Duplicado"],
    ["Recibo Duplicado"],
    default=vida_ajustes5['OBSERVACION']
)
vida_ajustes5 = vida_ajustes5.drop('NMRECIBO_DUPLICADO', axis=1)

# Separación de registros que se ajustan y los que quedan para revisión
vida_ajustados5 = vida_ajustes5.loc[vida_ajustes5['AJUSTAR?'] == "Ajustado Recibo Duplicado"]
vida_revision5 = vida_ajustes5.loc[vida_ajustes5['AJUSTAR?'] != "Ajustado Recibo Duplicado"]

#### Concatenación de Pólizas para Revisión y Ajustes Realizados

In [21]:
## Generación de tabla vida nuevamente con registros ajustados y registros a revisar

vida = pd.concat([vida_revision5, vida_ajustados1, vida_ajustados2, vida_ajustados3, vida_ajustados4, vida_ajustados5])
vida = vida.sort_values(by=['NMPOLIZA', 'NMRECIBO'], ascending=[True, True])
vida.reset_index(drop=True, inplace=True)

# Edición columna Observación para marcar polizas 99999999 y 999999 como revisión
vida['OBSERVACION'] = np.where((vida['NMRECIBO'].fillna("") == "99999999") | (vida['NMRECIBO'].fillna("") == "999999"),"Revisar",vida['OBSERVACION'])


#### Columnas SIGNO,ABS,CANTIDAD

In [22]:
# Columna Signo y asignación de valores positivos y negativos 
vida['SIGNO'] = np.sign(vida['PTCOMISION'])
vida['POSITIVO'] = np.where(vida['SIGNO'] == 1, vida['PTCOMISION'], 0).astype(float)
vida['NEGATIVO'] = np.where(vida['SIGNO'] == -1, vida['PTCOMISION'], 0).astype(float)

# Calculo de absoluto de la comisión
vida['ABS'] = abs(vida['PTCOMISION'])

# Contar la cantidad de cada póliza/recibo
vida_cantidad = vida.groupby(['NMPOLIZA', 'CDPROVISION'])['PTCOMISION'].count().reset_index(name='CANTIDAD')
vida = vida.merge(vida_cantidad, on=['NMPOLIZA', 'CDPROVISION'], how='left')

#### Identificación de Tipo de Registro

In [23]:
# Asignarle un tipo a cada registro que no vaya a ser ajustado
vida['TIPO'] = np.select(
    [vida['CDTIPO_FUENTE'].isin(provision),
     vida['CDTIPO_FUENTE'].isin(pago)],
    ["Provision",
     "Pago"],
    default="Ajuste"
)

# Agrupar por Poliza y contar las ocurrencias de cada valor en Tipo
vida_tipo = vida[vida['TIPO'] != ""]
vida_tipo = vida_tipo.groupby('NMPOLIZA')['TIPO'].unique().reset_index(name='UNICOS')
vida_tipo['TIPO_UNICO'] = ""

for index, row in vida_tipo.iterrows():
    if len(row['UNICOS']) == 1: 
        vida_tipo.at[index, 'TIPO_UNICO'] = f"Solo {row['UNICOS'][0]}"
    else:
        vida_tipo.at[index, 'TIPO_UNICO'] = ""

vida_tipo = vida_tipo.drop('UNICOS', axis=1)

# Cruce de tabla agrupada con la completa para identificar el tipo unico
vida = vida.merge(vida_tipo, on='NMPOLIZA', how='left')

#### Pólizas en Dólares

In [24]:
## Cruce de tabla vida con base de datos de pólizas en dolares
vida =  vida.merge(dolares, left_on='NMPOLIZA', right_on='NMPOLIZA', how='left')

## Agrupar por póliza/recibo y sumar valores positivos y negativos para calcular el % pendiente
porc_pendiente = vida.groupby(['NMPOLIZA', 'CDPROVISION']).agg({'NEGATIVO': 'sum','POSITIVO':'sum','PTCOMISION':'sum'}).reset_index()
porc_pendiente['%PENDIENTE'] = np.select(
    [porc_pendiente['POSITIVO'] == 0],
    [1],
    default=porc_pendiente['PTCOMISION'] / porc_pendiente['POSITIVO']
)
porc_pendiente = porc_pendiente[['NMPOLIZA', 'CDPROVISION', '%PENDIENTE']]

## Cruce de tabla agrupada con la completa para identificar el % pendiente
vida = vida.merge(porc_pendiente, on=['NMPOLIZA', 'CDPROVISION'], how='left')

## Marcar las pólizas en dólares que tienen porcentaje pendiente y son aptas para ajuste
vida['AJUSTAR?'] = np.select(
    [(vida['DOLARES'] == "X") & (vida['%PENDIENTE'] !=0) & (vida['%PENDIENTE'].between(desde_porc_pendientes, hasta_porc_pendientes, inclusive='both'))],
    ["Ajustado Dólares"],
    default=vida['AJUSTAR?']
)

vida['OBSERVACION'] = vida['OBSERVACION'].fillna("")

## Observación para las pólizas no dólares que tienen porcentaje pendiente
for index, row in vida.iterrows():
    if (row['DOLARES'] != "X") and (row['OBSERVACION'] != "") and (row['%PENDIENTE'] !=0) and (desde_porc_pendientes <= row['%PENDIENTE'] <= hasta_porc_pendientes):
        vida.at[index, 'OBSERVACION'] = f"{row['OBSERVACION']}, Porcentaje pendiente pequeño"
    elif (row['DOLARES'] != "X") and (row['OBSERVACION'] == "") and (row['%PENDIENTE'] !=0) and  (desde_porc_pendientes <= row['%PENDIENTE'] <= hasta_porc_pendientes):
        vida.at[index, 'OBSERVACION'] = f"Porcentaje pendiente pequeño"
    else:
        vida.at[index, 'OBSERVACION'] = row['OBSERVACION']

# Tabla para identificar el valor Ajustes Financieros por póliza en dólares
vida_ajustados6 = vida.loc[vida['AJUSTAR?'] == "Ajustado Dólares"]

#### Códigos Directos

In [25]:
## Cruce de tabla vida con base de datos de códigos directos
vida =  vida.merge(codigos_directos, left_on='CDAGENTE', right_on='CDAGENTE', how='left')

#### Asesores Retirados

In [26]:
## Cruce de tabla vida con base de datos de asesores retirados
vida = vida.merge(retirados, left_on='CDAGENTE', right_on='CDAGENTE', how='left')
vida['FEREGISTRO'] = pd.to_datetime(vida['FEREGISTRO'])

## Identificar si un asesor tiene registros luego de su fecha de retiro
vida['POST_RETIRO'] = np.select(
    [(vida['RETIRADOS'] == "X") & (vida['FEREGISTRO'] > vida['FEBAJA'])],
    ["Registro Post-Retiro"],
    default=""
)

## Calcular días entre la fecha de retiro del asesor y la fecha de registro de cada elemento
vida['DIAS_RETIRO'] = (vida['FEBAJA'] - vida['FEREGISTRO']).dt.days

#### Asesores FyD

In [27]:
## Cruce de tabla vida con base de datos de asesores en FyD
vida = vida.merge(asesores_formacion, left_on='CDAGENTE', right_on='CDAGENTE', how='left')

#### Comentario

In [28]:
vida['COMENTARIO'] = ""

#### Exportar Tabla con Depuraciones e Identificación de Ajustes

In [29]:
# Calcula el número total de filas en el DataFrame
total_filas = len(vida)

# Define el tamaño de la fracción
tamano_fraccion = 500000

# Calcula el número total de fracciones
total_fracciones = total_filas // tamano_fraccion + (1 if total_filas % tamano_fraccion != 0 else 0)

# Bucle para dividir y exportar en fracciones
for i in range(total_fracciones):
    inicio = i * tamano_fraccion
    fin = min((i + 1) * tamano_fraccion, total_filas)
    
    # Obtén la fracción actual del DataFrame
    fraccion_df = vida.iloc[inicio:fin]
    
    # Exporta la fracción a un archivo CSV
    fraccion_df.to_csv(f'02. Output/02. Vida/fraccion_{i + 1}_vida.csv', index=False, sep=';', encoding="latin1")

    print(f"Exportadas filas {inicio+1}-{fin} a fraccion_{i + 1}.csv")

Exportadas filas 1-500000 a fraccion_1.csv
Exportadas filas 500001-1000000 a fraccion_2.csv
Exportadas filas 1000001-1500000 a fraccion_3.csv
Exportadas filas 1500001-2000000 a fraccion_4.csv
Exportadas filas 2000001-2500000 a fraccion_5.csv
Exportadas filas 2500001-3000000 a fraccion_6.csv
Exportadas filas 3000001-3500000 a fraccion_7.csv
Exportadas filas 3500001-4000000 a fraccion_8.csv
Exportadas filas 4000001-4500000 a fraccion_9.csv
Exportadas filas 4500001-5000000 a fraccion_10.csv
Exportadas filas 5000001-5500000 a fraccion_11.csv
Exportadas filas 5500001-5804780 a fraccion_12.csv


#### Generar plantillas de Ajustes Financieros

In [30]:
# Tabla para identificar el valor Ajustes Financieros por póliza
vida_ajustados1['FUENTE'] = np.select(
    [vida_ajustados1['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

vida_plantilla_ajustes1 = vida_ajustados1.fillna("").groupby(['NMPOLIZA','CDRAMO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()
vida_plantilla_ajustes1['NMRECIBO'] = 99999999
vida_plantilla_ajustes1 = vida_plantilla_ajustes1[['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE','PTCOMISION']]

# Tabla para identificar el valor Ajustes Financieros por póliza/recibo
vida_ajustados2['FUENTE'] = np.select(
    [vida_ajustados2['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

vida_plantilla_ajustes2 = vida_ajustados2.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Tabla para identificar el valor Ajustes Financieros por póliza/agente
vida_ajustados3['FUENTE'] = np.select(
    [vida_ajustados3['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

vida_plantilla_ajustes3 = vida_ajustados3.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Tabla para identificar el valor Ajustes Financieros por código de agrupación
vida_ajustados4['FUENTE'] = np.select(
    [vida_ajustados4['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

vida_plantilla_ajustes4 = vida_ajustados4.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Tabla para identificar el valor Ajustes Financieros por recibo duplicado
vida_ajustados5['FUENTE'] = np.select(
    [vida_ajustados5['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

vida_plantilla_ajustes5 = vida_ajustados5.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

# Concatenar dataframes de ajustes (menos por póliza y pólizas en dólares)
vida_plantilla_ajustes1_40 = pd.concat([vida_plantilla_ajustes1,vida_plantilla_ajustes2, vida_plantilla_ajustes3, vida_plantilla_ajustes4, vida_plantilla_ajustes5], axis=0)

In [31]:
# Parametros para clave de contabilización 40
vida_plantilla_ajustes1_40['Agrupador'] = 1
vida_plantilla_ajustes1_40['Agrupador'] = vida_plantilla_ajustes1_40['Agrupador'].cumsum()
vida_plantilla_ajustes1_40['Fecha de Documento'] = ultimo_dia_del_mes
vida_plantilla_ajustes1_40['Clase de Documento'] = "SR"
vida_plantilla_ajustes1_40['Sociedad'] = "2000"
vida_plantilla_ajustes1_40['Fecha Contabilización'] = ultimo_dia_del_mes
vida_plantilla_ajustes1_40['Período'] = ultimo_dia_del_mes[2:4]
vida_plantilla_ajustes1_40['Clave de Moneda'] = "COP"
vida_plantilla_ajustes1_40['Tipo de Cambio'] = ""
vida_plantilla_ajustes1_40['Fe.conversión '] = ""
vida_plantilla_ajustes1_40['Número documento de referencia'] = vida_plantilla_ajustes1_40['NMRECIBO']
vida_plantilla_ajustes1_40['Texto de Cebecera'] = "AJUSTE PROVISION"
vida_plantilla_ajustes1_40['Calc. Impuestos '] = ""
vida_plantilla_ajustes1_40['Clave de contabilización para la siguiente posición'] = 40
vida_plantilla_ajustes1_40['Número de identificación fiscal 1'] = ""
vida_plantilla_ajustes1_40['Tipo de número de identificación fiscal'] = ""
vida_plantilla_ajustes1_40['In. CME'] = ""
vida_plantilla_ajustes1_40['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(vida_plantilla_ajustes1_40['PTCOMISION'] > 0) & (vida_plantilla_ajustes1_40['FUENTE'] == "GW"),
     (vida_plantilla_ajustes1_40['PTCOMISION'] < 0) & (vida_plantilla_ajustes1_40['FUENTE'] == "GW"),
     (vida_plantilla_ajustes1_40['PTCOMISION'] > 0) & (vida_plantilla_ajustes1_40['FUENTE'] == "CS"),
     (vida_plantilla_ajustes1_40['PTCOMISION'] < 0) & (vida_plantilla_ajustes1_40['FUENTE'] == "CS")],
    ["2481010040",
     "5209101010",
     "2481010000",
     "5209101000"],
    default="Revisar"
)
vida_plantilla_ajustes1_40['Importe en la moneda del documento'] = abs(vida_plantilla_ajustes1_40['PTCOMISION'].round(0)).astype(int)
vida_plantilla_ajustes1_40['Indicador IVA'] = ""
vida_plantilla_ajustes1_40['Indicador retefuente WT_WITHCD'] = ""
vida_plantilla_ajustes1_40['TP retefuente WITHT'] = ""
vida_plantilla_ajustes1_40['indicador reteica WT_WITHCD'] = ""
vida_plantilla_ajustes1_40['TP reteica WITHT'] = ""
vida_plantilla_ajustes1_40['indicador reteiva WT_WITHCD'] = ""
vida_plantilla_ajustes1_40['TP reteiva WITHT'] = ""
vida_plantilla_ajustes1_40['indicador retetimbre WT_WITHCD'] = ""
vida_plantilla_ajustes1_40['Tp retetimbre WITHT'] = ""
#######################################################################
division = pd.concat([vida_ajustados1,vida_ajustados2,vida_ajustados3,vida_ajustados4,vida_ajustados5], axis=0)[['NMPOLIZA','NMRECIBO','CDDELEGACION']].fillna("")
division = division.rename(columns={'CDDELEGACION':'División'})
vida_plantilla_ajustes1_40 = vida_plantilla_ajustes1_40.merge(division,on=['NMPOLIZA','NMRECIBO'], how='left')
vida_plantilla_ajustes1_40['Duplicado'] = vida_plantilla_ajustes1_40['Agrupador'].astype(str) + vida_plantilla_ajustes1_40['NMPOLIZA'] + vida_plantilla_ajustes1_40['NMRECIBO'].astype(str) + vida_plantilla_ajustes1_40['PTCOMISION'].astype(str) + vida_plantilla_ajustes1_40['Clave de contabilización para la siguiente posición'].astype(str)
vida_plantilla_ajustes1_40 = vida_plantilla_ajustes1_40.drop_duplicates(subset=['Duplicado'], keep='first')
vida_plantilla_ajustes1_40 = vida_plantilla_ajustes1_40.drop(['Duplicado'], axis=1)
vida_plantilla_ajustes1_40['División'] = np.select(
    [(vida_plantilla_ajustes1_40['División'].str.len() < 4),
     (vida_plantilla_ajustes1_40['División'] == "0000") | (vida_plantilla_ajustes1_40['División'].str.len() > 4)],
    [vida_plantilla_ajustes1_40['División'].str.rjust(4, "0"),
     "0099"],
    default=vida_plantilla_ajustes1_40['División']
)
#vida_plantilla_ajustes1_40['División'] = ""
#######################################################################
vida_plantilla_ajustes1_40['Clave de condiciones de pago'] = ""
vida_plantilla_ajustes1_40['Centro de coste'] = "20" + vida_plantilla_ajustes1_40['División'] + "000"
vida_plantilla_ajustes1_40['Centro de beneficio'] = ""
vida_plantilla_ajustes1_40['Días del descuento por pronto pago 1'] = ""
vida_plantilla_ajustes1_40['Porcentaje de descuento por pronto pago 1'] = ""
vida_plantilla_ajustes1_40['Días del descuento por pronto pago 2'] = ""
vida_plantilla_ajustes1_40['Porcentaje de descuento por pronto pago 2'] = ""
vida_plantilla_ajustes1_40['Plazo para condición de pago neto'] = ""
vida_plantilla_ajustes1_40['Fecha Contabilización'] = ultimo_dia_del_mes
vida_plantilla_ajustes1_40['Fecha base para cálculo del vencimiento'] = ""
vida_plantilla_ajustes1_40['Condición de pago fija'] = ""
vida_plantilla_ajustes1_40['Base de descuento'] = ""
vida_plantilla_ajustes1_40['Vía de pago'] = ""
vida_plantilla_ajustes1_40['Bloqueo pago'] = ""
vida_plantilla_ajustes1_40['% DPP'] = ""
vida_plantilla_ajustes1_40['Importe DPP'] = ""
vida_plantilla_ajustes1_40['Número de asignación'] = vida_plantilla_ajustes1_40['NMRECIBO']
vida_plantilla_ajustes1_40['Texto posición'] = "AJUSTE PROVISION"
vida_plantilla_ajustes1_40['Número de orden'] = ""
vida_plantilla_ajustes1_40['Clave de referencia de interlocutor comercial'] = vida_plantilla_ajustes1_40['NMPOLIZA']
vida_plantilla_ajustes1_40['Clave de referencia de interlocutor comercial2'] = vida_plantilla_ajustes1_40['CDRAMO']  
vida_plantilla_ajustes1_40['Clave de referencia para la posición de documento'] = vida_plantilla_ajustes1_40['CDAGENTE']
vida_plantilla_ajustes1_40['Tipo de banco interlocutor'] = ""
vida_plantilla_ajustes1_40['Fecha expedicion'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
vida_plantilla_ajustes1_40['Fecha fin vigencia'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
vida_plantilla_ajustes1_40['Poliza líder'] = ""
vida_plantilla_ajustes1_40['Cert.lider'] = ""
vida_plantilla_ajustes1_40['Nit'] = "8909037905"
vida_plantilla_ajustes1_40['Nombre'] = ""

# Parametros para clave de contabilización 50
vida_plantilla_ajustes1_50 = vida_plantilla_ajustes1_40.copy()
vida_plantilla_ajustes1_50['Clave de contabilización para la siguiente posición'] = 50
vida_plantilla_ajustes1_50['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(vida_plantilla_ajustes1_50['PTCOMISION'] > 0) & (vida_plantilla_ajustes1_50['FUENTE'] == "GW"),
     (vida_plantilla_ajustes1_50['PTCOMISION'] < 0) & (vida_plantilla_ajustes1_50['FUENTE'] == "GW"),
     (vida_plantilla_ajustes1_50['PTCOMISION'] > 0) & (vida_plantilla_ajustes1_50['FUENTE'] == "CS"),
     (vida_plantilla_ajustes1_50['PTCOMISION'] < 0) & (vida_plantilla_ajustes1_50['FUENTE'] == "CS")],
    ["5209101010",
     "2481010040",
     "5209101000",
     "2481010000"],
    default="Revisar"
)

# Concatenación de las dos tablas de clave de contabilización
vida_plantilla_ajustes1_final = pd.concat([vida_plantilla_ajustes1_40, vida_plantilla_ajustes1_50], axis=0)
vida_plantilla_ajustes1_final = vida_plantilla_ajustes1_final.drop(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE','PTCOMISION'], axis=1)
vida_plantilla_ajustes1_final = vida_plantilla_ajustes1_final.sort_values(by=['Agrupador', 'Clave de contabilización para la siguiente posición'])

In [33]:
# Calcula el número total de filas en el DataFrame
total_filas2 = len(vida_plantilla_ajustes1_final)

# Define el tamaño de la fracción
tamano_fraccion2 = 19998

# Calcula el número total de fracciones
total_fracciones2 = total_filas2 // tamano_fraccion2 + (1 if total_filas2 % tamano_fraccion2 != 0 else 0)

# Bucle para dividir y exportar en fracciones
for i in range(total_fracciones2):
    inicio2 = i * tamano_fraccion2
    fin2 = min((i + 1) * tamano_fraccion2, total_filas2)
    
    # Obtén la fracción actual del DataFrame
    fraccion_df2 = vida_plantilla_ajustes1_final.iloc[inicio2:fin2]
    
    # Exporta la fracción a un archivo CSV
    fraccion_df2.to_csv(f'02. Output/02. Vida/Ajustes/Ajustes Financieros {i + 1}.csv', index=False, sep=';', encoding="latin1")

    print(f"Exportadas filas {inicio2+1}-{fin2} a fraccion_{i + 1}.csv")

Exportadas filas 1-19998 a fraccion_1.csv
Exportadas filas 19999-39996 a fraccion_2.csv
Exportadas filas 39997-59994 a fraccion_3.csv
Exportadas filas 59995-79992 a fraccion_4.csv
Exportadas filas 79993-99990 a fraccion_5.csv
Exportadas filas 99991-119988 a fraccion_6.csv
Exportadas filas 119989-139986 a fraccion_7.csv
Exportadas filas 139987-159984 a fraccion_8.csv
Exportadas filas 159985-179982 a fraccion_9.csv
Exportadas filas 179983-199980 a fraccion_10.csv
Exportadas filas 199981-219978 a fraccion_11.csv
Exportadas filas 219979-239976 a fraccion_12.csv
Exportadas filas 239977-259974 a fraccion_13.csv
Exportadas filas 259975-279972 a fraccion_14.csv
Exportadas filas 279973-299970 a fraccion_15.csv
Exportadas filas 299971-319968 a fraccion_16.csv
Exportadas filas 319969-339966 a fraccion_17.csv
Exportadas filas 339967-359964 a fraccion_18.csv
Exportadas filas 359965-379962 a fraccion_19.csv
Exportadas filas 379963-399960 a fraccion_20.csv
Exportadas filas 399961-419958 a fraccion_21

In [34]:
# Tabla para identificar el valor Ajustes Financieros por póliza en dólares
vida_ajustados6['FUENTE'] = np.select(
    [vida_ajustados6['CDTIPO_FUENTE'].str.contains("CORE")],
    ["GW"],
    default="CS"
)

vida_plantilla_ajustes2_40 = vida_ajustados6.fillna("").groupby(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE'])['PTCOMISION'].sum().reset_index()

In [35]:
# Parametros para clave de contabilización 40
vida_plantilla_ajustes2_40['Agrupador'] = 1
vida_plantilla_ajustes2_40['Agrupador'] = vida_plantilla_ajustes2_40['Agrupador'].cumsum()
vida_plantilla_ajustes2_40['Fecha de Documento'] = ultimo_dia_del_mes
vida_plantilla_ajustes2_40['Clase de Documento'] = "SR"
vida_plantilla_ajustes2_40['Sociedad'] = "1000"
vida_plantilla_ajustes2_40['Fecha Contabilización'] = ultimo_dia_del_mes
vida_plantilla_ajustes2_40['Período'] = ultimo_dia_del_mes[2:4]
vida_plantilla_ajustes2_40['Clave de Moneda'] = "USD"
vida_plantilla_ajustes2_40['Tipo de Cambio'] = ""
vida_plantilla_ajustes2_40['Fe.conversión '] = ""
vida_plantilla_ajustes2_40['Número documento de referencia'] = vida_plantilla_ajustes2_40['NMRECIBO']
vida_plantilla_ajustes2_40['Texto de Cebecera'] = "MONEDA"
vida_plantilla_ajustes2_40['Calc. Impuestos '] = ""
vida_plantilla_ajustes2_40['Clave de contabilización para la siguiente posición'] = 40
vida_plantilla_ajustes2_40['Número de identificación fiscal 1'] = ""
vida_plantilla_ajustes2_40['Tipo de número de identificación fiscal'] = ""
vida_plantilla_ajustes2_40['In. CME'] = ""
vida_plantilla_ajustes2_40['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(vida_plantilla_ajustes2_40['PTCOMISION'] > 0) & (vida_plantilla_ajustes2_40['FUENTE'] == "GW"),
     (vida_plantilla_ajustes2_40['PTCOMISION'] < 0) & (vida_plantilla_ajustes2_40['FUENTE'] == "GW"),
     (vida_plantilla_ajustes2_40['PTCOMISION'] > 0) & (vida_plantilla_ajustes2_40['FUENTE'] == "CS"),
     (vida_plantilla_ajustes2_40['PTCOMISION'] < 0) & (vida_plantilla_ajustes2_40['FUENTE'] == "CS")],
    ["2481010040",
     "5209101010",
     "2481010000",
     "5209101000"],
    default="Revisar"
)
vida_plantilla_ajustes2_40['Importe en la moneda del documento'] = abs(vida_plantilla_ajustes2_40['PTCOMISION'].round(0)).astype(int)
vida_plantilla_ajustes2_40['Indicador IVA'] = ""
vida_plantilla_ajustes2_40['Indicador retefuente WT_WITHCD'] = ""
vida_plantilla_ajustes2_40['TP retefuente WITHT'] = ""
vida_plantilla_ajustes2_40['indicador reteica WT_WITHCD'] = ""
vida_plantilla_ajustes2_40['TP reteica WITHT'] = ""
vida_plantilla_ajustes2_40['indicador reteiva WT_WITHCD'] = ""
vida_plantilla_ajustes2_40['TP reteiva WITHT'] = ""
vida_plantilla_ajustes2_40['indicador retetimbre WT_WITHCD'] = ""
vida_plantilla_ajustes2_40['Tp retetimbre WITHT'] = ""
#######################################################################
division2 = vida_ajustados6[['NMPOLIZA','NMRECIBO','CDDELEGACION']].fillna("")
division2 = division2.rename(columns={'CDDELEGACION':'División'})
vida_plantilla_ajustes2_40 = vida_plantilla_ajustes2_40.merge(division2,on=['NMPOLIZA','NMRECIBO'], how='left')
vida_plantilla_ajustes2_40['Duplicado'] = vida_plantilla_ajustes2_40['Agrupador'].astype(str) + vida_plantilla_ajustes2_40['NMPOLIZA'] + vida_plantilla_ajustes2_40['NMRECIBO'].astype(str) + vida_plantilla_ajustes2_40['PTCOMISION'].astype(str) + vida_plantilla_ajustes2_40['Clave de contabilización para la siguiente posición'].astype(str)
vida_plantilla_ajustes2_40 = vida_plantilla_ajustes2_40.drop_duplicates(subset=['Duplicado'], keep='first')
vida_plantilla_ajustes2_40 = vida_plantilla_ajustes2_40.drop(['Duplicado'], axis=1)
vida_plantilla_ajustes2_40['División'] = np.select(
    [(vida_plantilla_ajustes2_40['División'].str.len() < 4),
     (vida_plantilla_ajustes2_40['División'] == "0000") | (vida_plantilla_ajustes2_40['División'].str.len() > 4)],
    [vida_plantilla_ajustes2_40['División'].str.rjust(4, "0"),
     "0099"],
    default=vida_plantilla_ajustes2_40['División']
)
#generales_plantilla_ajustes1_40['División'] = ""
#######################################################################
vida_plantilla_ajustes2_40['Clave de condiciones de pago'] = ""
vida_plantilla_ajustes2_40['Centro de coste'] = "10" + vida_plantilla_ajustes2_40['División'] + "000"
vida_plantilla_ajustes2_40['Centro de beneficio'] = ""
vida_plantilla_ajustes2_40['Días del descuento por pronto pago 1'] = ""
vida_plantilla_ajustes2_40['Porcentaje de descuento por pronto pago 1'] = ""
vida_plantilla_ajustes2_40['Días del descuento por pronto pago 2'] = ""
vida_plantilla_ajustes2_40['Porcentaje de descuento por pronto pago 2'] = ""
vida_plantilla_ajustes2_40['Plazo para condición de pago neto'] = ""
vida_plantilla_ajustes2_40['Fecha Contabilización'] = ultimo_dia_del_mes
vida_plantilla_ajustes2_40['Fecha base para cálculo del vencimiento'] = ""
vida_plantilla_ajustes2_40['Condición de pago fija'] = ""
vida_plantilla_ajustes2_40['Base de descuento'] = ""
vida_plantilla_ajustes2_40['Vía de pago'] = ""
vida_plantilla_ajustes2_40['Bloqueo pago'] = ""
vida_plantilla_ajustes2_40['% DPP'] = ""
vida_plantilla_ajustes2_40['Importe DPP'] = ""
vida_plantilla_ajustes2_40['Número de asignación'] = vida_plantilla_ajustes2_40['NMRECIBO']
vida_plantilla_ajustes2_40['Texto posición'] = "AJUSTE PROVISION"
vida_plantilla_ajustes2_40['Número de orden'] = ""
vida_plantilla_ajustes2_40['Clave de referencia de interlocutor comercial'] = vida_plantilla_ajustes2_40['NMPOLIZA']
vida_plantilla_ajustes2_40['Clave de referencia de interlocutor comercial2'] = vida_plantilla_ajustes2_40['CDRAMO']  
vida_plantilla_ajustes2_40['Clave de referencia para la posición de documento'] = vida_plantilla_ajustes2_40['CDAGENTE']
vida_plantilla_ajustes2_40['Tipo de banco interlocutor'] = ""
vida_plantilla_ajustes2_40['Fecha expedicion'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
vida_plantilla_ajustes2_40['Fecha fin vigencia'] = ultimo_dia_del_mes[0:2] + "." + ultimo_dia_del_mes[2:4] + "." + ultimo_dia_del_mes[4:]
vida_plantilla_ajustes2_40['Poliza líder'] = ""
vida_plantilla_ajustes2_40['Cert.lider'] = ""
vida_plantilla_ajustes2_40['Nit'] = "8909034079"
vida_plantilla_ajustes2_40['Nombre'] = ""

# Parametros para clave de contabilización 50
vida_plantilla_ajustes2_50 = vida_plantilla_ajustes2_40.copy()
vida_plantilla_ajustes2_50['Clave de contabilización para la siguiente posición'] = 50
vida_plantilla_ajustes2_50['Cuenta de mayor de la contabilidad principal'] = np.select(
    [(vida_plantilla_ajustes2_50['PTCOMISION'] > 0) & (vida_plantilla_ajustes2_50['FUENTE'] == "GW"),
     (vida_plantilla_ajustes2_50['PTCOMISION'] < 0) & (vida_plantilla_ajustes2_50['FUENTE'] == "GW"),
     (vida_plantilla_ajustes2_50['PTCOMISION'] > 0) & (vida_plantilla_ajustes2_50['FUENTE'] == "CS"),
     (vida_plantilla_ajustes2_50['PTCOMISION'] < 0) & (vida_plantilla_ajustes2_50['FUENTE'] == "CS")],
    ["5209101010",
     "2481010040",
     "5209101000",
     "2481010000"],
    default="Revisar"
)

# Concatenación de las dos tablas de clave de contabilización
vida_plantilla_ajustes2_final = pd.concat([vida_plantilla_ajustes2_40, vida_plantilla_ajustes2_50], axis=0)
vida_plantilla_ajustes2_final = vida_plantilla_ajustes2_final.drop(['NMPOLIZA','CDRAMO','NMRECIBO','CDAGENTE','FUENTE','PTCOMISION'], axis=1)
vida_plantilla_ajustes2_final = vida_plantilla_ajustes2_final.sort_values(by=['Agrupador', 'Clave de contabilización para la siguiente posición'])

In [36]:
# Calcula el número total de filas en el DataFrame
total_filas3 = len(vida_plantilla_ajustes2_final)

# Define el tamaño de la fracción
tamano_fraccion3 = 19998

# Calcula el número total de fracciones
total_fracciones3 = total_filas3 // tamano_fraccion3 + (1 if total_filas3 % tamano_fraccion3 != 0 else 0)

# Bucle para dividir y exportar en fracciones
for i in range(total_fracciones3):
    inicio3 = i * tamano_fraccion3
    fin3 = min((i + 1) * tamano_fraccion3, total_filas3)
    
    # Obtén la fracción actual del DataFrame
    fraccion_df3 = vida_plantilla_ajustes2_final.iloc[inicio3:fin3]
    
    # Exporta la fracción a un archivo CSV
    fraccion_df3.to_csv(f'02. Output/02. Vida/Ajustes Dólares/Ajustes Financieros {i + 1}.csv', index=False, sep=';', encoding="latin1")

    print(f"Exportadas filas {inicio3+1}-{fin3} a fraccion_{i + 1}.csv")

Exportadas filas 1-286 a fraccion_1.csv
